In [345]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
import hashlib
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

scope = ["playlist-modify-private","playlist-read-private","user-library-modify","user-library-read"]

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run ../src/useful_functions.ipynb
%run ../src/spotify_scraping.ipynb
%run ../src/spotify_scraping_playlists.ipynb

In [346]:
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()
library = get_spotify_library()

In [354]:
#get_library(max=10000)


offset = 0
total = 1000
temp = pd.DataFrame()

while offset<total:
    temp_playlists = spotify.current_user_playlists(limit=50, offset=offset)
    temp = pd.concat([temp,pd.DataFrame.from_dict({'Playlist ID': [x['id'] for x in temp_playlists['items']],
                                                   'Playlist Name': [x['name'] for x in temp_playlists['items']],
                                                   'Playlist Description': [x['description'] for x in temp_playlists['items']]})])
    if offset == 0:
        total = temp_playlists['total']
    offset += 50

del(offset, total, temp_playlists)

playlists_to_run = list(temp[[((x == '**') or (x[0:6] == '** ')) for x in temp['Playlist Description']]]['Playlist ID']) 
for playlist_id in tqdm(playlists_to_run):
    get_playlist(playlist_id, with_tracks = True)
    
playlists_to_run = list(temp[[((x == '*****') or (x[0:6] == '***** ')) for x in temp['Playlist Description']]]['Playlist ID']) 
for playlist_id in tqdm(playlists_to_run):
    get_playlist(playlist_id, with_tracks = True)

del(temp, playlists_to_run)

100%|██████████| 9/9 [01:45<00:00, 11.76s/it]


In [348]:
temp2 = tracks.merge(library, on='Track ID', how='left',indicator = True)
temp2 = temp2[temp2['Is Track Local'] == False]
temp2 = temp2.merge(temp2[temp2['_merge'] == 'both'][["Track Name","Track ID", "Artist ID's"]], on = ['Track Name', "Artist ID's"], how='left', suffixes=[""," Mapped"])
temp2['Track ID Mapped'] = temp2.apply(lambda x: x['Track ID'] if pd.isna(x['Track ID Mapped']) else x['Track ID Mapped'], axis=1)
temp2 = temp2.groupby(['Track ID', 'Track Name']).agg('count')
temp2 = temp2[temp2['Track ID Mapped'] > 1].sort_values(by=['Track Name']).reset_index()

#del(temp2)

In [56]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

,Track ID,Playlist ID,Position Number
0,2TfDppfA0KZ3rTQT0dgrTo,07B5mpuLVdVpgSycsqn4Qc,0
0,6As82n96RAwuoR9Y69N0EB,07B5mpuLVdVpgSycsqn4Qc,1
0,32mPZhhxoVDEMy32hk6lu3,07B5mpuLVdVpgSycsqn4Qc,2
0,5aMiCg40vZKkzFedNs2p7K,07B5mpuLVdVpgSycsqn4Qc,3
0,2fmtrXfvbN8xkHwE0zgTTc,07B5mpuLVdVpgSycsqn4Qc,4
...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,607
0,4f5SAiUVc4WFvuxva3JhnR,07B5mpuLVdVpgSycsqn4Qc,608
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,609
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,610


In [344]:
artists['Artist ID'] = artists.apply(lambda x: x['Artist ID'] if not pd.isna(x['Artist ID']) else 'local' + hash_(x['Artist Name']), axis = 1)


('0ey6crPk5fXQWkq21iPTMV',
 '5h6KJPKB8cSVJTWZhKAZoT',
 '2z6598dYz4eQbCJ1t0kT1N',
 '5DD5GZd4ElmQTy9NleMvKJ',
 '1sLpCr5xThZkGHLw4H0mNE',
 '1arQYOWtxuQW27WR299gT3',
 '38EQ4PdxziEDJOpwFHSb5c',
 '4R024vmmAv1tyJGZPWkEDu',
 '6rbA09kzdZICoLg7sWbkGd',
 '6ULjGlnhcCwHIJigN47K5U',
 '3dGzchO8ZLLvTt5CA3BRlN',
 '4EB1DygdDurhbqV9qEBJkg',
 '1iA1G6TFhNc3GerD6e5XNm',
 '4BTddjd5LzqjQ5WAViSw0g',
 '2g6KJkh4hrSTIjzp2hd5aY',
 '2cxi12PeZmsDkSlJi1slRp',
 '1Lu14VE1i2ga6W93mIGfWA',
 '1fvjrL59qNJVJKNQB8RUg3',
 '1lK8wUX7lIy1cchRWfQic6',
 '6wSxVh97cc5ZTSXvPgvv81',
 '6AuRHPIlYDhz1UA0GOkhjm',
 '6oYWNIrMewoS8GBkN8f58f',
 '6l3iPTof4l2D75Wawq9w7R',
 '25zUD40u8M2kJmdcabBzrz',
 '5qEhKFhIXtSGi9Gw449LKX',
 '2wn2nqzITvJ1vcMRO8Wzv6',
 '3g6hHqzYmzgrqXh1KnRbLK',
 '5kdYrM3h2sB1Eid5tDf6Hk',
 '5UTYnSzQ9STJudZpDZvmk3',
 '67pLmA3GVpK4P0bq11rQqa',
 '75kWLz33mwSCBminHvUY9I',
 '3mNygoyrEKLgo6sx0MzwOL',
 '2nCACYdIndYchzX4bxLcTW',
 '0oKYiTD5CdNbrofRvM1dIr',
 '3U6lOF5scy8TdKoZjfBh0o',
 '4ziDIu5XdfA2rQpEAvs0BN',
 '22qVoqkMA3F9iviFEx4W5e',
 

In [332]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')
library.to_pickle('../data/spotify_library.pkl')